# CS/ECE/ISyE 524 - Spr 2021 - HW 9
### Ben Hayum

# 1. Making Change

How should you make change for 99 cents if the goal is to minimize the total weight of the coins used? You may use any number of each type of coin. Here are the weights of each coin:

|Type of Coin       | penny | nickel | dime | quarter |
|:------------------|-------|--------|------|---------|
|Weight             |  2.500|5.000   | 2.268| 5.670   |



Print the number of each type of coin used, and the total weight.

In [1]:
# problem data
weights = [ 2.500, 5.000, 2.268, 5.670]
values = [ 1, 5, 10, 25 ]
n = length(weights) # total number of coin types

using JuMP, Cbc
m = Model(Cbc.Optimizer)

#using JuMP, HiGHS
#m = Model(HiGHS.Optimizer)

@variable(m, coin_binary[1:4, 1:7], Bin)
@variable(m, 0 <= coin_count[1:4] <= 100)

# Using binary to get integers for the coin counts
for i in 1:4
    @constraint(m, coin_count[i] == 1*coin_binary[i, 1] + 2*coin_binary[i, 2] + 4*coin_binary[i, 3] + 
                                8*coin_binary[i, 4] + 16*coin_binary[i, 5] + 32*coin_binary[i, 6] + 
                                64*coin_binary[i, 7])
end

@constraint(m, sum(coin_count[i]*values[i] for i in 1:n) == 99 )
@objective(m, Min, sum(coin_count[i]*weights[i] for i in 1:n) )

optimize!(m)

println("Total coin weight: ", objective_value(m))
println("\nNumber of pennies: ", value(coin_count[1]))
println("Number of nickels: ", value(coin_count[2]))
println("Number of dimes: ", value(coin_count[3]))
println("Number of quarters: ", value(coin_count[4]))
println("\n")

Total coin weight: 31.54600000000001

Number of pennies: 4.0
Number of nickels: 0.0
Number of dimes: 6.999999999999999
Number of quarters: 1.0


Welcome to the CBC MILP Solver 
Version: 2.10.5 
Build Date: Mar 11 2021 

command line - Cbc_C_Interface -solve -quit (default strategy 1)
Continuous objective value is 22.4532 - 0.00 seconds
Cgl0004I processed model has 1 rows, 18 columns (18 integer (18 of which binary)) and 18 elements
Cbc0038I Initial state - 1 integers unsatisfied sum - 0.3
Cbc0038I Pass   1: suminf.    0.12000 (1) obj. 22.4532 iterations 1
Cbc0038I Pass   2: suminf.    0.45000 (1) obj. 56.474 iterations 1
Cbc0038I Pass   3: suminf.    0.45000 (1) obj. 56.474 iterations 0
Cbc0038I Pass   4: suminf.    0.42500 (1) obj. 56.474 iterations 1
Cbc0038I Pass   5: suminf.    0.42500 (1) obj. 56.474 iterations 0
Cbc0038I Pass   6: suminf.    0.42500 (1) obj. 56.474 iterations 0
Cbc0038I Pass   7: suminf.    0.38750 (1) obj. 75.804 iterations 3
Cbc0038I Pass   8: suminf.    0.3875

# 2. Comquat Computers

Comquat owns four production plants at which personal computers are produced. Comquat can sell up to 20,000 computers per year at a price of \$3,500 per computer. For each plant the production capacity, cost per computer, and fixed cost of operating the plant for a year are given below. Determine how Comquat can maximize its yearly profit from computer production.

| Plant | Production capacity | Plant fixed cost (\$ Million) | Cost per computer (\$) |
|:-----:|:-------------------:|:-----------------------------:|:----------------------:|
| 1     | 10,000              |   9                           |  1,000                 |
| 2     |  8,000              |   5                           |  1,700                 |
| 3     |  9,000              |   3                           |  2,300                 |
| 4     |  6,000              |   1                           |  2,900                 |

In [2]:
using JuMP, HiGHS
m = Model(HiGHS.Optimizer)
set_silent(m)

price     = 3.5e3                       # price at which we can sell a single computer 
                                        # (regardless of where it is produced)
cap       = 20000                       # maximum computers that can be sold per year

capacity  = [10000, 8000, 9000, 6000]   # production capacity for each plant
fixedcost = [9e6, 5e6, 3e6, 1e6]        # fixed cost for each plant
itemcost  = [1e3, 1.7e3, 2.3e3, 2.9e3]  # cost per computer for each factory


# insert your code here, with integer variable vector z and real variable vector x
@variable(m, z[1:4], Bin) # will the plant be open or closed
@variable(m, x[1:4] >= 0) # number of computers produced at each plant

@constraint(m, sum(x) <= 20000)
for i in 1:4
    @constraint(m, x[i] <= capacity[i])
    @constraint(m, x[i] <= cap*z[i])
end

@objective(m, Max, sum(x)*price - sum(x[i]*itemcost[i] for i in 1:4) - sum(z[i]*fixedcost[i] for i in 1:4) )

optimize!(m)
println("The maximal profit is: ", objective_value(m))
println("Here is a tally of factory number, whether it's open or not, and how many computers it should produce.")
value.([1:4 z x])

The maximal profit is: 2.56e7
Here is a tally of factory number, whether it's open or not, and how many computers it should produce.


4×3 Matrix{Float64}:
 1.0  1.0  10000.0
 2.0  1.0   8000.0
 3.0  0.0      0.0
 4.0  1.0   2000.0

# 3. ABC Investments

ABC Inc. is considering several investment options.  Each option has a minimum investment required as well as a maximum investment allowed. These restrictions, along with the expected return are summarized in the following table (figures are in millions of dollars):

|Option | Minimum investment | Maximum investment | Expected return (%)  |
|:-----:|:------------------:|:------------------:|:--------------------:|
|  1    |      3             |      27            |     13               |
|  2    |      2             |      12            |      9               |
|  3    |      9             |      35            |     17               |
|  4    |      5             |      15            |     10               |
|  5    |     12             |      46            |     22               |
|  6    |      4             |      18            |     12               |

Because of the high-risk nature of Option 5, company policy requires that the total amount invested in Option 5 be no more that the combined amount invested in Options 2, 4 and 6.  In addition, if an investment is made in Option 3, it is required that at least a minimum investment be made in Option 6.  ABC has \$80 million to invest and obviously wants to maximize its total expected return on investment.  Which options should ABC invest in, and how much should be invested?

In [3]:
using JuMP, HiGHS
m = Model(HiGHS.Optimizer)
set_silent(m)

lows  = [  3,  2,  9,  5, 12,  4 ]  # minimum investment
highs = [ 27, 12, 35, 15, 46, 18 ]  # maximum investment
ret   = [ 13,  9, 17, 10, 22, 12 ]  # expected return

money = 80

@variable(m, z[1:6], Bin)
@variable(m, invest[1:6] >= 0)

@constraint(m, sum(invest) <= money)
@constraint(m, invest[5] <= invest[2] + invest[4] + invest[6]) 
@constraint(m, invest[6] >= z[3]*lows[6]) 
for i in 1:6
    @constraint(m, invest[i] >= z[i]*lows[i])
    @constraint(m, invest[i] <= z[i]*highs[i])
end

@objective(m, Max, sum(ret[i]*invest[i]/100 for i in 1:6))
optimize!(m)

# Check these numbers

println("The maximum return on investment (net profit) is: \$", objective_value(m), " million.")
println("This is an average return of: ", 100*objective_value(m)/value(sum(invest)), "%")
println("Here is a tally of each investment, whether we invest or not, and how much we invest (in millions of \$).")
value.([1:6 z invest])

The maximum return on investment (net profit) is: $13.5 million.
This is an average return of: 16.875%
Here is a tally of each investment, whether we invest or not, and how much we invest (in millions of $).


6×3 Matrix{Float64}:
 1.0  0.0   0.0
 2.0  0.0   0.0
 3.0  1.0  35.0
 4.0  1.0   5.0
 5.0  1.0  22.5
 6.0  1.0  17.5

# 4. Paint production

As part of its weekly production, a paint company produces five batches of paints, always the same, for some big clients who have a stable demand.  Every paint batch is produced in a single production process, all in the same blender that needs to be cleaned between each batch.  The durations of blending paint batches 1 to 5 are 40, 35, 45, 32 and 50 minutes respectively.  The cleaning times depend of the colors and the paint types.  For example, a long cleaning period is required if an oil-based paint is produced after a water-based paint, or to produce white paint after a dark color.  The times are given in minutes in the following matrix $A$ where $A_{ij}$ denotes the cleaning time after batch $i$ if it is followed by batch $j$.

$$
A = \begin{bmatrix}
 0&11& 7&13&11 \\
 5& 0&13&15&15 \\
13&15& 0&23&11 \\
 9&13& 5& 0& 3 \\
 3& 7& 7& 7& 0
\end{bmatrix}
$$

Since the company has other activities, it wishes to deal with this weekly production in the shortest possible time (blending and cleaning).  What is the corresponding order of paint batches?  The order will be applied every week, so the cleaning time between the last batch of one week and the first of the following week needs to be accounted for in the total duration of cleaning.
                

In [4]:
# HELPER FUNCTION: returns the cycle containing the city START.
batches = 1:5

function getSubtour(x,start)
    subtour = [start]
    while true
        j = subtour[end]
        for k in batches
            if x[k,j] == 1
                push!(subtour,k)
                break
            end
        end
        if subtour[end] == start
            break
        end
    end
    return subtour
end

# HELPER FUNCTION: returns a list of all cycles
function getAllSubtours(x)
    nodesRemaining = batches
    subtours = []
    while length(nodesRemaining) > 0
        subtour = getSubtour(x,nodesRemaining[1])
        push!(subtours, subtour)
        nodesRemaining = setdiff(nodesRemaining,subtour)
    end
    return subtours
end


getAllSubtours (generic function with 1 method)

In [5]:
#(Hint: This problem is a nonsymmetric traveling salesman problem — “nonsymmetric” because the
# distance between nodes i and j differs according to the direction of travel.)
# 2 of 2


# A[i,j] is the time it takes to clean after batch i if followed by batch j
A = [  0  11   7  13  11
       5   0  13  15  15
      13  15   0  23  11
       9  13   5   0   3
       3   7   7   7   0 ]

# time it takes to blend a particular batch. 
b = [ 40, 35, 45, 32, 50 ]

using JuMP, HiGHS
m = Model(HiGHS.Optimizer)
set_silent(m)

n = length(b)  # number of paints

# YOUR CODE HERE
@variable(m, x[1:n,1:n], Bin)                                       # must formulate as IP this time
@constraint(m, c1[j in 1:n], sum( x[i,j] for i in 1:n ) == 1)     # one out-edge
@constraint(m, c2[i in 1:n], sum( x[i,j] for j in 1:n ) == 1)     # one in-edge
@constraint(m, c3[i in 1:n], x[i,i] == 0 )                        # no self-loops
@objective(m, Min, sum( x[i,j]*A[i,j] for i in 1:n, j in 1:n ))   # minimize total cost

sols = []

for iters = 1:30
    optimize!(m)
    println("Tour length: ", objective_value(m))
    xx = value.(x)
    push!(sols,xx)
    subtours = getAllSubtours(xx)  # get all the subtours
    display(subtours)
    len = length(subtours)
    if len == 1  # solution is just a single tour!
        println("(The blends go in the reverse direction of the array outputted above)\n")
        println("SOLVED!\n")
        break
    else
        for subtour in subtours
            L = length(subtour)
            @constraint(m, sum( x[subtour[k+1],subtour[k]] for k = 1:L-1 ) <= L-2)
            @constraint(m, sum( x[subtour[k],subtour[k+1]] for k = 1:L-1 ) <= L-2)
        end
    end
end
X = value.(x);
for i = 1:n
    for j=1:n
        if (X[i,j] >= .99)
            println(" blend ",i," is followed by blend ",j," requiring cleanup time ",A[i,j])
        end
    end
end

println("and the minimum cleanup time is: ", objective_value(m), " min, (plus ", sum(b), " min of blending)")

Tour length: 37.0


2-element Vector{Any}:
 [1, 2, 3, 1]
 [4, 5, 4]

Tour length: 39.0


2-element Vector{Any}:
 [1, 2, 1]
 [3, 4, 5, 3]

Tour length: 41.0


1-element Vector{Any}:
 [1, 2, 5, 3, 4, 1]

(The blends go in the reverse direction of the array outputted above)

SOLVED!

 blend 1 is followed by blend 4 requiring cleanup time 13
 blend 2 is followed by blend 1 requiring cleanup time 5
 blend 3 is followed by blend 5 requiring cleanup time 11
 blend 4 is followed by blend 3 requiring cleanup time 5
 blend 5 is followed by blend 2 requiring cleanup time 7
and the minimum cleanup time is: 41.0 min, (plus 202 min of blending)
